# 10000stepの10回分の平均座標からドット積を求める

### モジュールのインストール

In [2]:
#from vpython import *
#GlowScript 3.0 VPython
import vpython as vp
import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import math
import numpy as np
import csv
import scipy as sp
from scipy.optimize import curve_fit
import math
import numexpr as ne

<IPython.core.display.Javascript object>

### キャプションの追加

In [3]:
vs.scene.caption = """<b>3D text can be "billboard" text -- always facing you.</b>
Note that the "Regular text" has different colors on the front, back and sides.
To rotate "camera", drag with right button or Ctrl-drag.
To zoom, drag with middle button or Alt/Option depressed, or use scroll wheel.
  On a two-button mouse, middle is left + right.
To pan left/right and up/down, Shift-drag.
Touch screen: pinch/extend to zoom, swipe or two-finger rotate."""
   
vp.scene.background = vp.color.white

NameError: name 'vs' is not defined

### 座標に名前をつける

In [ ]:
names = ['step']
for i in range(1,12):
    names+=['x'+str(i),'y'+str(i),'z'+str(i)]
#print(names)
data = pd.read_csv("my_output.txt",sep=" ",index_col=0,header = None,names=names )

#index_col=0 →indexとして使いたい列の列番号を0始まりで指定する。
#特定の列だけを読み込む場合、引数usecolsを使う。
#カラム名を指定したいときはnames引数で指定
#csvファイルの中身を読み込んだDataFrameまたはテキストパーサーが返される。

## timestepが10000における10回分の平均座標を求める

### 読み込んだデータの確認

In [ ]:
data

### タイムステップが9999のときだけ取り出す

In [ ]:
data.loc[9999]
#loc：行と列のラベル名を指定して1つの要素、もしくは範囲を指定して複数の要素を参照する

### ドット積を求める

In [ ]:
all_products = []
for line in data.loc[9999].to_numpy():
    temp = []
    x0 = line[3:6]-line[0:3]
    x0 /= np.linalg.norm(x0)
    for index in range(int(len(line)/3)-1):
        xn = line[index*3+3:index*3+6]-line[index*3:index*3+3]
        xn /= np.linalg.norm(xn) #numpy.linalg.norm(x) xはノルムの値を求めるための入力配列
        temp.append(np.dot(x0,xn)) #np.dot ベクトルの内積を求める
    all_products.append(temp)
all_products = np.array(all_products)

In [ ]:
avg = all_products.sum(axis=0)/10.0 
#np.sum(axis = NONE):ndarrayの全要素を足し合わせる
#axis:どの軸方向に要素を足し合わせていくかを指定。Noneの時は全ての要素を足し合わせる。

In [ ]:
print(avg)
print(avg[0])
print(avg[9])

In [ ]:
plt.plot(avg)
#plt.yscale('log')
plt.show()

### ドット積の結果をファイルに出力し、読み込む

In [ ]:
with open("dot_average.txt", mode='w') as f:
    for num in range(10):       
        f.write(str(num) + " " + str(avg[num])+ "\n")
f.close()
data2 = pd.read_csv("dot_average.txt",sep = " ",header = None, index_col = 0)

### 持続長の定義

In [ ]:
def exponential(x,a,b):
    return a*np.exp(-x / b)

### シミュレーション結果を用いて、持続長の公式における最適な係数を求める

In [ ]:
popt, pcov = curve_fit(exponential, data2.index.to_numpy(), data2[1])
#poptは最適推定値、pcovは共分散
#curve_fit(f, xdata, ydata, p0=None, sigma=None, absolute_sigma=False, check_finite=True, bounds=- inf, inf, method=None, jac=None, **kwargs)
print(popt[0]) #aの値
print(popt[1]) #bの値

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, xlabel='k', ylabel='<nkn0>')

ax.plot(data2[1], "r", label = "Simulation data")
ax.plot(exponential(data2.index.to_numpy(),*popt), "b", label = "Exponential fit: " + str(round(popt[0],2)) + "exp(-k/" + str(round(popt[1],2))+ ")")
ax.plot(xlabel='k', ylabel='$<n_k$・$n_0>$')
ax.legend(bbox_to_anchor=(1, 1), loc='upper right', borderaxespad=0, fontsize=10)
fig.savefig('persistence_sample.png')

### ボクセル の表示

In [ ]:
#for c in range(len(data)-10, len(data)):
#print(data3)
for c in range(0, len(data)):
    x1,y1,z1 = 200.0*data.iloc[c][0:3] #iloc : 複数の要素の値を指定してデータを取り出す
    x2,y2,z2 = 200.0*data.iloc[c][3:6]
    x3,y3,z3 = 200.0*data.iloc[c][6:9] 
    x4,y4,z4 = 200.0*data.iloc[c][9:12] 
    x5,y5,z5 = 200.0*data.iloc[c][12:15]
    x6,y6,z6 = 200.0*data.iloc[c][15:18] 
    x7,y7,z7 = 200.0*data.iloc[c][18:21] 
    x8,y8,z8 = 200.0*data.iloc[c][21:24]
    x9,y9,z9 = 200.0*data.iloc[c][24:27] 
    x10,y10,z10 = 200.0*data.iloc[c][27:30]
    x11,y11,z11 = 200.0*data.iloc[c][30:33]
     
    vs.box(pos=vs.vec(x1,y1,z1), color=vs.color.blue)
    c1,c2,c3 = cm.summer(c/10000.0)[:3]
    t1,t2,t3 = cm.autumn(c/10000.0)[:3]
    
    vp.box(pos=vp.vec(x2,y2,z2), color=vp.vec(c1,c2,c3))
    vp.box(pos=vp.vec(x3,y3,z3), color=vp.vec(t1, t2, t3))
    vp.box(pos=vp.vec(x4,y4,z4), color=vp.vec(c1,c2,c3))
    vp.box(pos=vp.vec(x5,y5,z5), color=vp.vec(t1, t2, t3))
    vp.box(pos=vp.vec(x6,y6,z6), color=vp.vec(c1, c2, c3))
    vp.box(pos=vp.vec(x7,y7,z7), color=vp.vec(t1, t2, t3))
    vp.box(pos=vp.vec(x8,y8,z8), color=vp.vec(c1,c2,c3))
    vp.box(pos=vp.vec(x9,y9,z9), color=vp.vec(t1, t2, t3))
    vp.box(pos=vp.vec(x10,y10,z10), color=vp.vec(c1, c2, c3))
    vp.box(pos=vp.vec(x11,y11,z11), color=vp.vec(t1, t2, t3))